# Проект по определению эмоциональной тоннальности комментариев

<h1>Содержание<span class="tocSkip"></span></h1>
</ul></li><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Подключение-необходимых-бибилиотек-и-модулей" data-toc-modified-id="Подключение-необходимых-бибилиотек-и-модулей-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Подключение необходимых бибилиотек и модулей</a></span></li><li><span><a href="#Функции-для-более-удобной-работы" data-toc-modified-id="Функции-для-более-удобной-работы-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Функции для более удобной работы</a></span></li><li><span><a href="#Токенизация-и-лемматизация" data-toc-modified-id="Токенизация-и-лемматизация-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Токенизация и лемматизация</a></span><ul class="toc-item"><li><span><a href="#Вывод:" data-toc-modified-id="Вывод:-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Вывод:</a></span></li></ul></li><li><span><a href="#Разделение-на-обучающую-и-тестовую-выборки:" data-toc-modified-id="Разделение-на-обучающую-и-тестовую-выборки:-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Разделение на обучающую и тестовую выборки:</a></span></li><li><span><a href="#Устранение-дисбаланса-целевого-признака-на-обучающей-выборке" data-toc-modified-id="Устранение-дисбаланса-целевого-признака-на-обучающей-выборке-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Устранение дисбаланса целевого признака на обучающей выборке</a></span></li><li><span><a href="#Вывод:" data-toc-modified-id="Вывод:-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Вывод:</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Векторизация-данных-и-обучение-модели-логистической-регрессии" data-toc-modified-id="Векторизация-данных-и-обучение-модели-логистической-регрессии-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Векторизация данных и обучение модели логистической регрессии</a></span></li><li><span><a href="#Применение-NGRAM-в-процессе-обучения" data-toc-modified-id="Применение-NGRAM-в-процессе-обучения-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Применение NGRAM в процессе обучения</a></span></li><li><span><a href="#Использование-pipeline-и-кросс-валидации" data-toc-modified-id="Использование-pipeline-и-кросс-валидации-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Использование pipeline и кросс-валидации</a></span></li><li><span><a href="#LinearSVC" data-toc-modified-id="LinearSVC-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>LinearSVC</a></span></li><li><span><a href="#RandomForestClassifier" data-toc-modified-id="RandomForestClassifier-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>RandomForestClassifier</a></span></li><li><span><a href="#LGBMClassifier" data-toc-modified-id="LGBMClassifier-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>LGBMClassifier</a></span></li><li><span><a href="#Вывод:" data-toc-modified-id="Вывод:-2.7"><span class="toc-item-num">2.7&nbsp;&nbsp;</span>Вывод:</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

## Подготовка

### Подключение необходимых бибилиотек и модулей

In [1]:
import pandas as pd
import numpy as np
import re 
from scipy.stats import randint as sp_randint

import nltk
from nltk.corpus import stopwords as nltk_stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier

from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline
from sklearn.utils import shuffle
from sklearn.metrics import f1_score

from IPython.core.display import HTML
from IPython.core.interactiveshell import InteractiveShell

# Задаем опцию отображения результатов расчетов в ячейках
pd.options.mode.chained_assignment = None

### Функции для более удобной работы

Подключим функцию для отображения датафреймов: 

In [2]:
def df_preview(df,number_of_rows=-1):
    if number_of_rows==-1:
        display(HTML(df.to_html()))
    else:    
        display(HTML(df.head(number_of_rows).to_html()))

Загрузка и изучение данных

In [3]:
df=pd.read_csv(r'/datasets/toxic_comments.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


Посмотрим на содержимое датафрейма:

In [5]:
df_preview(df, 10)

,text,toxic
0,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",0
1,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",0
2,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",0
3,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of """"types of accidents"""" -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know.\n\nThere appears to be a backlog on articles for review so I guess there may be a delay until a reviewer turns up. It's listed in the relevant form eg Wikipedia:Good_article_nominations#Transport """,0
4,"You, sir, are my hero. Any chance you remember what page that's on?",0
5,"""\n\nCongratulations from me as well, use the tools well. · talk """,0
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,"Your vandalism to the Matt Shirvington article has been reverted. Please don't do it again, or you will be banned.",0
8,"Sorry if the word 'nonsense' was offensive to you. Anyway, I'm not intending to write anything in the article(wow they would jump on me for vandalism), I'm merely requesting that it be more encyclopedic so one can use it for school as a reference. I have been to the selective breeding page but it's almost a stub. It points to 'animal breeding' which is a short messy article that gives you no info. There must be someone around with expertise in eugenics? 93.161.107.169",0
9,alignment on this subject and which are contrary to those of DuLithgow,0


Есть подозрения, что целевой признак имеет неравномерное распределение. Проверим данные на наличие дисбаланса: 

In [6]:
df['toxic'].value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

Наши подозрения оправдались. Количество положительных ответов меньше числа отрицательных ответов примерно в 10 раз. Что говорит о сильном дисбалансе - после разделения данных на выборки, нам нужно будет провести масштабирование обучающей выборки. 

### Токенизация и лемматизация

Переведем наши тексты в списки состоящие из отдельных слов и приведем их к нормальной форме. Для чего напишем функцию: 

In [7]:
def text_processing(text):
    # Преобразуем текст к нижнему регистру
    text=text.lower() 
    # Удалим ненужные символы
    text=re.sub(r'[^a-zA-Z ]', ' ', text) 
    # Уберем лишние пробелы
    text=text.split() 
    text=' '.join(text)

    # Токенизируем и лемматизируем текст + сформируем список лемматизированных слов
    stemmer = SnowballStemmer(language="english")
    tokens = word_tokenize(text)
    lemm_list=[]
    for token in tokens:
        lemm_list.append(stemmer.stem(token))

    # Превратим полученный список обратно в текст    
    text=''
    for word in lemm_list:            
        text+=word+' '
            
    return text

Для наглядного отображения текста добавим в наш датафрейм столбец копию, чтобы потом сравнить "До" и "После":

In [8]:
%%time
df['oldtext']=df['text']
df['text']=df['text'].apply(text_processing)

CPU times: user 3min 55s, sys: 585 ms, total: 3min 55s
Wall time: 3min 57s


In [9]:
df=df[['oldtext','text','toxic']]

Посмотрим на данные, которые у нас получились: 

In [10]:
df_preview(df,10)

,oldtext,text,toxic
0,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",explan whi the edit made under my usernam hardcor metallica fan were revert they weren t vandal just closur on some gas after i vote at new york doll fac and pleas don t remov the templat from the talk page sinc i m retir now,0
1,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",d aww he match this background colour i m seem stuck with thank talk januari utc,0
2,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",hey man i m realli not tri to edit war it s just that this guy is constant remov relev inform and talk to me through edit instead of my talk page he seem to care more about the format than the actual info,0
3,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of """"types of accidents"""" -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know.\n\nThere appears to be a backlog on articles for review so I guess there may be a delay until a reviewer turns up. It's listed in the relevant form eg Wikipedia:Good_article_nominations#Transport """,more i can t make ani real suggest on improv i wonder if the section statist should be later on or a subsect of type of accid i think the refer may need tidi so that they are all in the exact same format ie date format etc i can do that later on if no one els doe first if you have ani prefer for format style on refer or want to do it yourself pleas let me know there appear to be a backlog on articl for review so i guess there may be a delay until a review turn up it s list in the relev form eg wikipedia good articl nomin transport,0
4,"You, sir, are my hero. Any chance you remember what page that's on?",you sir are my hero ani chanc you rememb what page that s on,0
5,"""\n\nCongratulations from me as well, use the tools well. · talk """,congratul from me as well use the tool well talk,0
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,cocksuck befor you piss around on my work,1
7,"Your vandalism to the Matt Shirvington article has been reverted. Please don't do it again, or you will be banned.",your vandal to the matt shirvington articl has been revert pleas don t do it again or you will be ban,0
8,"Sorry if the word 'nonsense' was offensive to you. Anyway, I'm not intending to write anything in the article(wow they would jump on me for vandalism), I'm merely requesting that it be more encyclopedic so one can use it for school as a reference. I have been to the selective breeding page but it's almost a stub. It points to 'animal breeding' which is a short messy article that gives you no info. There must be someone around with expertise in eugenics? 93.161.107.169",sorri if the word nonsens was offens to you anyway i m not intend to write anyth in the articl wow they would jump on me for vandal i m mere request that it be more encycloped so one can use it for school as a refer i have been to the select breed page but it s almost a stub it point to anim breed which is a short messi articl that give you no info there must be someon around with expertis in eugen,0
9,alignment on this subject and which are contrary to those of DuLithgow,align on this subject and which are contrari to those of dulithgow,0


#### Вывод: 

Как видно из приведенных выше данных, наши тексты разбиты на отдельные слова, переведены к нижнему регистру, очищенны от ненужных и мусорных знаков и символов. Также все слова приведены к нормальной форме(лемматизированы). 

### Разделение на обучающую и тестовую выборки: 

Выделим целевой и обучающие признаки: 

In [11]:
features = df.drop(['toxic'], axis=1)
target = df['toxic']

In [12]:
features_train, features_test, target_train, target_test = train_test_split(features
                                                                           ,target
                                                                           ,test_size=.25
                                                                           ,random_state=12345
                                                                           ,stratify=target)

### Устранение дисбаланса целевого признака на обучающей выборке

Для устранения дисбаланса опробуем две техники upsempling и downsampling после чего в процессе обучения моделей оценим их эффективность и выберем лучшую для нашей задачи: 

In [13]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    target_zeros = target[target == 0]
    print('Количество отзывов с нетоксичными описаниями до уменьшения выборки:',len(features_zeros))
        
    features_ones = features[target == 1]
    target_ones = target[target == 1]
    print('Количество отзывов с токсичными описаниями до уменьшения выборки:',len(features_ones))

    features_downsampled = pd.concat([features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat([target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    print()
    
    features_zeros = features_downsampled[target_downsampled == 0]
    print('Количество отзывов с нетоксичными описаниями после уменьшения выборки:',len(features_zeros))          
          
    features_ones = features_downsampled[target_downsampled == 1]
    print('Количество отзывов с токсичными описаниями до уменьшения выборки:',len(features_ones))
    
    features_downsampled, target_downsampled = shuffle(features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

In [14]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    target_zeros = target[target == 0]
    print('Количество отзывов с нетоксичными описаниями до увеличения выборки:',len(features_zeros))
    
    features_ones = features[target == 1]
    target_ones = target[target == 1]
    print('Количество отзывов с токсичными описаниями до увеличения выборки:',len(features_ones))

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    print()
          
    features_zeros = features_upsampled[target_upsampled == 0]
    print('Количество отзывов с нетоксичными описаниями после увеличения выборки:',len(features_zeros))          
          
    features_ones = features_upsampled[target_upsampled == 1]
    print('Количество отзывов с токсичными описаниями после увеличения выборки:',len(features_ones))         
    
    features_upsampled, target_upsampled = shuffle(features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

Применим описанные выше функции к обучающей выборке: 

In [15]:
features_train_downsample,target_train_downsample=downsample(features_train,target_train,0.1132)

Количество отзывов с нетоксичными описаниями до уменьшения выборки: 107509
Количество отзывов с токсичными описаниями до уменьшения выборки: 12169

Количество отзывов с нетоксичными описаниями после уменьшения выборки: 12170
Количество отзывов с токсичными описаниями до уменьшения выборки: 12169


In [16]:
features_train_upsample,target_train_upsample=upsample(features_train,target_train,9)

Количество отзывов с нетоксичными описаниями до увеличения выборки: 107509
Количество отзывов с токсичными описаниями до увеличения выборки: 12169

Количество отзывов с нетоксичными описаниями после увеличения выборки: 107509
Количество отзывов с токсичными описаниями после увеличения выборки: 109521


### Вывод: 
Мы произвели обработку и подготовили данные к использованию в моделях машинного обучения.

## Обучение

### Векторизация данных и обучение модели логистической регрессии

Для начала создадим итоговый датафрейм, для оценки эффективности моделей ML, куда будем заносить результаты по каждой из них, для последующего сравнения

In [17]:
df_Result = pd.DataFrame({'Модель':[], 'Метрика F1':[]})

Проведем непосредственно векторизацию, для этого загрузим список английских стоп-слов:  

In [18]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Оценим качество модели логистической регрессии на выборке сбалансированной техникой downsampling

In [19]:
features_train = features_train_downsample
target_train = target_train_downsample

Переведем тексты в векторный вид: 

In [20]:
%%time
count_tf_idf = TfidfVectorizer(stop_words=stopwords)
tf_idf_model = count_tf_idf.fit_transform(features_train['text'])
tf_idf_train = count_tf_idf.transform(features_train['text'])
tf_idf_test = count_tf_idf.transform(features_test['text'])

CPU times: user 5.31 s, sys: 24 ms, total: 5.33 s
Wall time: 5.34 s


Обучим логистическую регрессию на выборке и оценим метрику качества: 

In [21]:
%%time
model = LogisticRegression(solver='lbfgs', max_iter=200, class_weight='balanced', random_state=777)
model.fit(tf_idf_train, target_train)
predict_test = model.predict(tf_idf_test)
f1_score_test = f1_score(target_test, predict_test) 
print('Точность модели на тестовой выборке:', round(f1_score_test,3))

Точность модели на тестовой выборке: 0.691
CPU times: user 8.62 s, sys: 9.28 s, total: 17.9 s
Wall time: 17.9 s


Метрика F1 оказалась не достаточной для прохождения заданного в задаче порога) 

Запишем результат в итоговую таблицу: 

In [22]:
df_Result=df_Result.append({
                            'Модель':'LogisticRegression (downsample)',
                            'Метрика F1':round(f1_score_test,3)
                           }
                           ,ignore_index=True)

Проверим метрику на данных сбалансированных с помощью upsampling

In [23]:
features_train = features_train_upsample
target_train = target_train_upsample

In [24]:
%%time
count_tf_idf = TfidfVectorizer(stop_words=stopwords)
tf_idf_model = count_tf_idf.fit_transform(features_train['text'])
tf_idf_train = count_tf_idf.transform(features_train['text'])
tf_idf_test = count_tf_idf.transform(features_test['text'])

CPU times: user 26.1 s, sys: 292 ms, total: 26.4 s
Wall time: 26.9 s


In [25]:
%%time
model = LogisticRegression(solver='lbfgs', max_iter=200, class_weight='balanced', random_state=777)
model.fit(tf_idf_train, target_train)
predict_test = model.predict(tf_idf_test)
f1_score_test = f1_score(target_test, predict_test) 
print('Точность модели на тестовой выборке:', round(f1_score_test,3))

Точность модели на тестовой выборке: 0.764
CPU times: user 49.4 s, sys: 41.5 s, total: 1min 30s
Wall time: 1min 30s


Как видно из приведенных выше данных - значение F1-меры значительно увеличилось. Модель преодолевает порог нашей задачи. Попробуем обучить модель на данных сиспользованием биграмм.

### Применение NGRAM в процессе обучения

In [26]:
%%time
count_tf_idf = TfidfVectorizer(ngram_range=(1, 2), stop_words=stopwords)
tf_idf_model = count_tf_idf.fit_transform(features_train['text'])
tf_idf_train = count_tf_idf.transform(features_train['text'])
tf_idf_test = count_tf_idf.transform(features_test['text'])

CPU times: user 1min 8s, sys: 844 ms, total: 1min 9s
Wall time: 1min 11s


In [27]:
%%time
model = LogisticRegression(solver='lbfgs', max_iter=200, random_state=777)
model.fit(tf_idf_train, target_train)
predict_test = model.predict(tf_idf_test)
f1_score_test = f1_score(target_test, predict_test) 
print('Точность модели на тестовой выборке:', round(f1_score_test,3))

Точность модели на тестовой выборке: 0.768
CPU times: user 1min 40s, sys: 49.9 s, total: 2min 30s
Wall time: 2min 30s


При применении NGRAM метрика качества незначительно увеличилась. Запишем полученный результат в итоговую таблицу: 

In [28]:
df_Result=df_Result.append({
                            'Модель':'LogisticRegression (upsample + ngram)',
                            'Метрика F1':round(f1_score_test,3)
                           }
                           ,ignore_index=True)

### Использование pipeline и кросс-валидации

Ппоробуем использовать pipeline и кросс- валидацию для повышения значения F1^

In [29]:
%%time
vect = TfidfVectorizer(ngram_range=(1, 2), stop_words=stopwords)
tfidf = TfidfTransformer()
clf = LogisticRegression(solver='lbfgs', max_iter=200, random_state=777)

pipeline = Pipeline([
    ('vect',vect),
    ('tfidf',tfidf),
    ('clf',clf)
])

scores = cross_val_score(pipeline, features_train['text'], target_train, cv=3, scoring='f1_micro')
pipeline.fit(features_train['text'],target_train)
predict_test = pipeline.predict(features_test['text'])
f1_score_test = f1_score(target_test, predict_test) 
print('Точность модели на тестовой выборке:', round(f1_score_test,3))

Точность модели на тестовой выборке: 0.753
CPU times: user 6min 5s, sys: 1min 43s, total: 7min 48s
Wall time: 7min 51s


Результат получился неожиданным, значение метрики уменьшилось. Добавим полученные данные в результирующую таблицу:

In [30]:
df_Result=df_Result.append({
                            'Модель':'LogisticRegression (upsample + ngram + кросс-валидация)',
                            'Метрика F1':round(f1_score_test,3)
                           }
                           ,ignore_index=True)

Попробуем другие модели ML, 

### LinearSVC

In [31]:
%%time
vect = TfidfVectorizer(stop_words=stopwords)
tfidf = TfidfTransformer()
clf = LinearSVC(max_iter=200, random_state=777)

pipeline = Pipeline([
    ('vect',vect),
    ('tfidf',tfidf),
    ('clf',clf)
])

scores = cross_val_score(pipeline, features_train['text'], target_train, cv=3, scoring='f1_micro')
pipeline.fit(features_train['text'],target_train)
predict_test = pipeline.predict(features_test['text'])
f1_score_test = f1_score(target_test, predict_test) 
print('Точность модели на тестовой выборке:', round(f1_score_test,3))

Точность модели на тестовой выборке: 0.735
CPU times: user 1min 3s, sys: 0 ns, total: 1min 3s
Wall time: 1min 3s


Результат - не фонтан. Запишем  его в таблицу:

In [32]:
df_Result=df_Result.append({
                            'Модель':'LinearSVC (upsample + ngram + кросс-валидация)',
                            'Метрика F1':round(f1_score_test,3)
                           }
                           ,ignore_index=True)

### RandomForestClassifier

In [33]:
%%time
vect = TfidfVectorizer(stop_words=stopwords)
tfidf = TfidfTransformer()
clf = RandomForestClassifier(random_state=777, n_estimators=100, max_depth=20, min_samples_split=7, n_jobs=-1)

pipeline = Pipeline([
    ('vect',vect),
    ('tfidf',tfidf),
    ('clf',clf)
])

scores = cross_val_score(pipeline, features_train['text'], target_train, cv=3, scoring='f1_micro')
pipeline.fit(features_train['text'],target_train)
predict_test = pipeline.predict(features_test['text'])
f1_score_test = f1_score(target_test, predict_test) 
print('Точность модели на тестовой выборке:', round(f1_score_test,3))

Точность модели на тестовой выборке: 0.408
CPU times: user 2min 8s, sys: 0 ns, total: 2min 8s
Wall time: 2min 9s


Да, модель случайного леса - явно не для этой задачи. Запишем итоговые значения в результирующую таблицу.

In [34]:
df_Result=df_Result.append({
                            'Модель':'RandomForestClassifier (upsample + ngram + кросс-валидация)',
                            'Метрика F1':round(f1_score_test,3)
                           }
                           ,ignore_index=True)

### LGBMClassifier

In [35]:
%%time
vect = TfidfVectorizer(stop_words=stopwords)
tfidf = TfidfTransformer()
clf = LGBMClassifier(random_state=777, n_jobs=-1)

pipeline = Pipeline([
    ('vect',vect),
    ('tfidf',tfidf),
    ('clf',clf)
])

scores = cross_val_score(pipeline, features_train['text'], target_train, cv=3, scoring='f1_micro')
pipeline.fit(features_train['text'],target_train)
predict_test = pipeline.predict(features_test['text'])
f1_score_test = f1_score(target_test, predict_test) 
print('Точность модели на тестовой выборке:', round(f1_score_test,3))

Точность модели на тестовой выборке: 0.746
CPU times: user 17min 49s, sys: 692 ms, total: 17min 50s
Wall time: 18min


Почти дотянулись до заветной планки. Но "почти" это не про машинное обучение. Запишем результат в таблицу:

In [36]:
df_Result=df_Result.append({
                            'Модель':'LGBMClassifier (upsample + ngram + кросс-валидация)',
                            'Метрика F1':round(f1_score_test,3)
                           }
                           ,ignore_index=True)

### Вывод:

Мы опробовали различные модели машинного обучения для определения тоанльности текстов. Собрали полученные данные в итоговую таблицу. Пришло время анализа результатов. 

## Выводы

Выведем итоговую таблицу для определния попбедитей в нашем соревномании моделей машинного обучения: 

In [37]:
df_preview(df_Result.sort_values(by='Метрика F1',ascending=False))

,Модель,Метрика F1
1,LogisticRegression (upsample + ngram),0.768
2,LogisticRegression (upsample + ngram + кросс-валидация),0.753
5,LGBMClassifier (upsample + ngram + кросс-валидация),0.746
3,LinearSVC (upsample + ngram + кросс-валидация),0.735
0,LogisticRegression (downsample),0.691
4,RandomForestClassifier (upsample + ngram + кросс-валидация),0.408


Как видно из таблицы - попбедителем в нашем соревновании становится **"Логистическая регрессия" с тюнингом в виде использования апсемблинга, TF-IDF и биграмм**.